# Lesson 04: Basic topographic features

*This lesson made by Simon M Mudd and last updated 22/11/2021*

In this lesson we are going to do some basic analysis of topography. There are a lot of different software tools for doing this, for example:
* [Whitebox](https://www.whiteboxgeo.com/download-whiteboxtools/)
* [TopoToolbox](https://topotoolbox.wordpress.com/)
* [SAGA](http://www.saga-gis.org/en/index.html)

However, for this example we will use [LSDTopoTools](https://github.com/LSDtopotools) because the person writing this lesson is also the lead developer of that software. The expectation for this lesson is that you are on a GeoSciences Notable notebook, where LSDTopoTools is already installed. If you aren't on that system you can also [install it on a Colab notebook](https://github.com/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools_on_colab.ipynb).

The objective of this practical is to give you a taster of what kinds of things you might do with topographic data. 

## First import some stuff we need

First we make sure lsdviztools version is updated (it needs to be > 0.4.7):

In [ ]:
!pip install lsdviztools --upgrade

Now we import a bunch of stuff

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Data preprocessing

For various historical reasons, **LSDTopoTools** does not read *GeoTiff* format, so we need to convert any rasters to [ENVI bil](https://www.l3harrisgeospatial.com/docs/enviimagefiles.html#:~:text=The%20ENVI%20image%20format%20is,an%20accompanying%20ASCII%20header%20file.&text=Band%2Dinterleaved%2Dby%2Dline,to%20the%20number%20of%20bands.) format. **This is not the same as ESRI bil! MAKE SURE YOU USE ENVI BIL!!**

You could convert any file to `ENVI bil` format using `gdalwarp` and then including the parametr `-of ENVI` (`of` stands for output format) but `lsdviztools` has some built in functions for doing that for you in python. 

We are going to use the ALOS W3D dataset (from the last lesson) for this lesson, and here is the conversion syntax:

In [ ]:
DataDirectory = "./"
RasterFile = "rio_aguas_AW3D30.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=30)

You can serach for specific files using the `!ls` command, so we can look for the file that has been created. 

There is a `.tif` file from the last lesson, but the two files with extensions `.bil` and `.hdr` are from the conversion. ENVI bil files have all the data in the `.bil` file and all the georeferencing in the `.hdr` file. The `.hdr` file is an ascii file so you can easily open these files in a text editor and see all the important metadata. 

In [ ]:
!ls rio_aguas_AW3D30_UTM*

# Now lets do some basic topographic analysis

Now will extract some topogaphic metrics using `lsdtopotools`. 

The `lsdtt_parameters` are the various parameters that you can use to run an analysis. We will discuss these later. For now, we will just follow this recipe. 

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "rio_aguas_AW3D30_UTM",
                                 write_prefix= "rio_aguas_AW3D30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

# Plot some data

We are now going to do some simple plots using a mapping package that we put together. There are more general ways to visualise data, but this makes pretty pictures quickly.  

In [ ]:
%matplotlib inline
Base_file = "rio_aguas_AW3D30_UTM"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=True, size_format="geomorphology", dpi=600)

In [ ]:
from IPython.display import Image
Image('rio_aguas_AW3D30_UTM_hillshade.png')

In [ ]:
Base_file = "rio_aguas_AW3D30_UTM"
Drape_prefix = "rio_aguas_AW3D30_UTM_SLOPE"
DataDirectory = "./"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, 
                              cmap = "bwr", cbar_loc = "right", 
                              cbar_label = "Gradient (m/m)",
                              save_fig=True, size_format="geomorphology",
                              colour_min_max = [0,1.25])

In [ ]:
from IPython.display import Image
Image('rio_aguas_AW3D30_UTM_drape.png')

# Get some channel profiles

Okay, we will now run a different analysis. We will get some channel profiles. 

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "rio_aguas_AW3D30_UTM",
                                 write_prefix= "rio_aguas_AW3D30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

We can look to see what files we have using the following command. the `!` tells this notebook to run a command on the underlying linux operating system, and `ls` in linux is a command to list files. 

In [ ]:
!ls

The file with the channels is the one with `chi_data_map` in the filename. We are going to load this into a `pandas` dataframe. You can think of `pandas` as a kind of excel for python. It does data handling of spreadsheet-like information (and loads more.)

In [ ]:
df = pd.read_csv("rio_aguas_SRTMGL1_UTM_chi_data_map.csv")
df.head()

In [ ]:
%matplotlib inline
fname_prefix = "rio_aguas_SRTMGL1_UTM"
ChannelFileName = "rio_aguas_SRTMGL1_UTM_chi_data_map.csv"
DataDirectory = "./"
lsdmw.PrintChiChannelsAndBasins(DataDirectory,fname_prefix, ChannelFileName, 
                                add_basin_labels = True, cmap = "jet", cbar_loc = "right", 
                                colorbarlabel = "elevation (m)", size_format = "ESURF", fig_format = "png", 
                                dpi = 400,plotting_column = "elevation")

Image('rio_aguas_SRTMGL1_UTM_chi_channels_and_basins.png')

In [ ]:
!ls *.png

That gives you a map of the basins that were extracted. 

You could also look at the channel profiles. 

## Picking your own drainage basins

What if you want to pick specific drainge basins?

You can do that by specifying the latitude and longitude of the basin outlets. 

You can go into google maps in a different browser window, right click 